<a href="https://colab.research.google.com/github/AliciaAPerez/Death_Analytics_using_CDC_Data_with_Machine_Learning/blob/alicia/Death_stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:13 https://developer.download.nvidia.com/compute/c

In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-04-20 13:38:33--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.30MB/s    in 0.7s    

2021-04-20 13:38:35 (1.30 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

In [ ]:
!curl -s https://course19.fast.ai/setup/colab | bash

Updating fastai...
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
Done.


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
#function for other data sets to be cleaned up - only can be used for 2014 and 2015 and newer, older pages use a different education marker
def data_clean_up(data):
  data = data.drop("education_1989_revision","education_reporting_flag","age_substitution_flag","age_recode_52","age_recode_27","age_recode_12", "infant_age_recode_22",
"place_of_death_and_decedents_status","method_of_disposition","autopsy", "activity_code", "place_of_injury_for_causes_w00_y34_except_y06_and_y07_", "358_cause_recode", "113_cause_recode",
"130_infant_cause_recode", "39_cause_recode", "bridged_race_flag", "race_imputation_flag", "race_recode_3", "race_recode_5", "hispanic_origin");
  data = data.filter((data.detail_age_type == 1))
  data = data.drop("detail_age_type")
  data = data.drop("number_of_entity_axis_conditions","entity_condition_4","entity_condition_5","entity_condition_6","entity_condition_7","entity_condition_8",
                                            "entity_condition_9","entity_condition_10","entity_condition_11","entity_condition_12","entity_condition_13","entity_condition_14",
                                            "entity_condition_15","entity_condition_16","entity_condition_17","entity_condition_18","entity_condition_19","entity_condition_20",
                                            "number_of_record_axis_conditions", "record_condition_1", "record_condition_2","record_condition_3","record_condition_4","record_condition_5",
                                            "record_condition_6","record_condition_7","record_condition_8","record_condition_9","record_condition_10","record_condition_11","record_condition_12",
                                            "record_condition_13","record_condition_14","record_condition_15","record_condition_16","record_condition_17","record_condition_18","record_condition_19",
                                            "record_condition_20",)
  data = data.withColumnRenamed("resident_status","resident").withColumnRenamed("education_2003_revision","education").withColumnRenamed("detail_age", "age").withColumnRenamed("current_date_year", "year").withColumnRenamed("icd_code_10th_revision", "icd_death_code").withColumnRenamed("hispanic_originrace_recode", "hispanic")
  data = data.withColumn("resident",col("resident").cast("integer")).withColumn("education",col("education").cast("integer")).withColumn("month_of_death",col("month_of_death").cast("integer")).withColumn("age",col("age").cast("integer")).withColumn("day_of_week_of_death",col("day_of_week_of_death").cast("integer")).withColumn("current_data_year",col("current_data_year").cast("integer")).withColumn("manner_of_death",col("manner_of_death").cast("integer")).withColumn("race",col("race").cast("integer")).withColumn("hispanic",col("hispanic").cast("integer"))
  data = data.fillna(value="0",subset=["entity_condition_1"]).fillna(value="0",subset=["entity_condition_2"]).fillna(value="0",subset=["entity_condition_3"])
  data = data.na.replace(["Y", "N", "U"], ["1","2","3"], "injury_at_work")
  data = data.na.replace(["M", "F"], ["1","2"], "sex")
  data = data.na.replace(["S", "M", "W", "D", "U"], ["1","2", "3", "4", "5"], "marital_status")
  data = data.select("icd_death_code","resident","education","month_of_death","sex", "age","marital_status","day_of_week_of_death","current_data_year","injury_at_work","manner_of_death","race","hispanic","entity_condition_1","entity_condition_2","entity_condition_3")
  data = data.na.drop()
  return data


In [ ]:
#function for other data sets to be cleaned up - only can be used for 2014 and 2015 and newer, older pages use a different education marker
def data_clean_up2(data):
  data = data.drop("education_2003_revision","education_reporting_flag","age_substitution_flag","age_recode_52","age_recode_27","age_recode_12", "infant_age_recode_22",
"place_of_death_and_decedents_status","method_of_disposition","autopsy", "activity_code", "place_of_injury_for_causes_w00_y34_except_y06_and_y07_", "358_cause_recode", "113_cause_recode",
"130_infant_cause_recode", "39_cause_recode", "bridged_race_flag", "race_imputation_flag", "race_recode_3", "race_recode_5", "hispanic_origin");
  data = data.filter((data.detail_age_type == 1))
  data = data.drop("detail_age_type")
  data = data.drop("number_of_entity_axis_conditions","entity_condition_4","entity_condition_5","entity_condition_6","entity_condition_7","entity_condition_8",
                                            "entity_condition_9","entity_condition_10","entity_condition_11","entity_condition_12","entity_condition_13","entity_condition_14",
                                            "entity_condition_15","entity_condition_16","entity_condition_17","entity_condition_18","entity_condition_19","entity_condition_20",
                                            "number_of_record_axis_conditions", "record_condition_1", "record_condition_2","record_condition_3","record_condition_4","record_condition_5",
                                            "record_condition_6","record_condition_7","record_condition_8","record_condition_9","record_condition_10","record_condition_11","record_condition_12",
                                            "record_condition_13","record_condition_14","record_condition_15","record_condition_16","record_condition_17","record_condition_18","record_condition_19",
                                            "record_condition_20",)
  data = data.withColumnRenamed("resident_status","resident").withColumnRenamed("education_1989_revision","education").withColumnRenamed("detail_age", "age").withColumnRenamed("current_date_year", "year").withColumnRenamed("icd_code_10th_revision", "icd_death_code").withColumnRenamed("hispanic_originrace_recode", "hispanic")
  data = data.withColumn("resident",col("resident").cast("integer")).withColumn("education",col("education").cast("integer")).withColumn("month_of_death",col("month_of_death").cast("integer")).withColumn("age",col("age").cast("integer")).withColumn("day_of_week_of_death",col("day_of_week_of_death").cast("integer")).withColumn("current_data_year",col("current_data_year").cast("integer")).withColumn("manner_of_death",col("manner_of_death").cast("integer")).withColumn("race",col("race").cast("integer")).withColumn("hispanic",col("hispanic").cast("integer"))
  data = data.fillna(value="0",subset=["entity_condition_1"]).fillna(value="0",subset=["entity_condition_2"]).fillna(value="0",subset=["entity_condition_3"])
  data = data.na.replace([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,99], [1,1,1,1,1,1,1,1,1,2,2,2,3,4,4,4,6,7,9], "education")
  data = data.na.replace(["Y", "N", "U"], ["1","2","3"], "injury_at_work")
  data = data.na.replace(["M", "F"], ["1","2"], "sex")
  data = data.na.replace(["S", "M", "W", "D", "U"], ["1","2", "3", "4", "5"], "marital_status")
  data = data.select("icd_death_code","resident","education","month_of_death","sex", "age","marital_status","day_of_week_of_death","current_data_year","injury_at_work","manner_of_death","race","hispanic","entity_condition_1","entity_condition_2","entity_condition_3")
  data = data.na.drop()
  return data

In [8]:
#GATHER DATA
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2015_data.csv"
spark.sparkContext.addFile(url)
death_2015 = spark.read.csv(SparkFiles.get("2015_data.csv"), sep=",", header=True)

# Show DataFrame
death_2015_final = data_clean_up(death_2015)
death_2015_final.show()

+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|icd_death_code|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|race|hispanic|entity_condition_1|entity_condition_2|entity_condition_3|
+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|          I500|       1|        3|             1|  1| 84|             2|                   1|             2015|             3|              7|   1|       6|            11I500|            61L031|                 0|
|          C349|       1|        6|             1|  1| 70|             2|                   2|             2015|             3|             

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2014_data.csv"
spark.sparkContext.addFile(url)
death_2014 = spark.read.csv(SparkFiles.get("2014_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2014_final = data_clean_up(death_2014)
death_2014_final.show()

+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|icd_death_code|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|race|hispanic|entity_condition_1|entity_condition_2|entity_condition_3|
+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|           I64|       1|        2|             1|  1| 87|             2|                   4|             2014|             3|              7|   1|       6|             11I64|                 0|                 0|
|          I250|       1|        2|             1|  1| 58|             4|                   3|             2014|             3|             

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2013_data.csv"
spark.sparkContext.addFile(url)
death_2013 = spark.read.csv(SparkFiles.get("2013_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2013_final = data_clean_up2(death_2013)
death_2013_final.show()

+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|icd_death_code|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|race|hispanic|entity_condition_1|entity_condition_2|entity_condition_3|
+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|           F03|       1|        4|             1|  1| 90|             2|                   4|             2013|             3|              7|   1|       6|             11F03|             61I10|            62K579|
|           F03|       1|        3|             1|  2| 80|             3|                   3|             2013|             3|             

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2012_data.csv"
spark.sparkContext.addFile(url)
death_2012 = spark.read.csv(SparkFiles.get("2012_data.csv"), sep=",", header=True)
death_2012 = death_2012.withColumnRenamed("icd_code_10","icd_death_code")
#run the function for clean_up and show that dataframe
death_2012_final = data_clean_up2(death_2012)
death_2012_final.show()

+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|icd_death_code|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|race|hispanic|entity_condition_1|entity_condition_2|entity_condition_3|
+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|          C349|       1|        3|             1|  2| 59|             3|                   1|             2012|             3|              7|   1|       6|            11C349|                 0|                 0|
|           F03|       1|        3|             1|  2| 90|             3|                   2|             2012|             3|             

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2011_data.csv"
spark.sparkContext.addFile(url)
death_2011 = spark.read.csv(SparkFiles.get("2011_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2011_final = data_clean_up2(death_2011)
death_2011_final.show()

+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|icd_death_code|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|race|hispanic|entity_condition_1|entity_condition_2|entity_condition_3|
+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|          C859|       2|        3|             1|  2| 65|             2|                   2|             2011|             3|              7|   1|       6|            11C859|                 0|                 0|
|          C509|       1|        1|             1|  2| 57|             1|                   7|             2011|             3|             

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2010_data.csv"
spark.sparkContext.addFile(url)
death_2010 = spark.read.csv(SparkFiles.get("2010_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2010_final = data_clean_up2(death_2010)
death_2010_final.show()

+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|icd_death_code|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|race|hispanic|entity_condition_1|entity_condition_2|entity_condition_3|
+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|          G049|       2|        3|             1|  2| 68|             4|                   6|             2010|             3|              7|   3|       8|            11G049|             61F03|                 0|
|           C37|       1|        1|             1|  2| 12|             1|                   1|             2010|             3|             

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2009_data.csv"
spark.sparkContext.addFile(url)
death_2009 = spark.read.csv(SparkFiles.get("2009_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2009_final = data_clean_up2(death_2009)
death_2009_final.show()

+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|icd_death_code|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|race|hispanic|entity_condition_1|entity_condition_2|entity_condition_3|
+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|          F329|       1|        3|             1|  2| 80|             2|                   2|             2009|             3|              7|  68|       8|             11E46|            21F329|             61I64|
|          C920|       1|        3|             1|  1| 77|             3|                   1|             2009|             3|             

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2008_data.csv"
spark.sparkContext.addFile(url)
death_2008 = spark.read.csv(SparkFiles.get("2008_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2008_final = data_clean_up2(death_2008)
death_2008_final.show()

+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|icd_death_code|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|race|hispanic|entity_condition_1|entity_condition_2|entity_condition_3|
+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|          C159|       1|        3|             1|  1| 65|             2|                   1|             2008|             3|              7|   1|       6|            11C159|                 0|                 0|
|          I251|       1|        2|             1|  1| 78|             2|                   7|             2008|             3|             

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2007_data.csv"
spark.sparkContext.addFile(url)
death_2007 = spark.read.csv(SparkFiles.get("2007_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2007_final = data_clean_up2(death_2007)
death_2007_final.show()

+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|icd_death_code|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|race|hispanic|entity_condition_1|entity_condition_2|entity_condition_3|
+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|          C259|       3|        1|             1|  1| 72|             2|                   2|             2007|             3|              7|   1|       6|            11C259|                 0|                 0|
|           X74|       1|        3|             1|  1| 35|             2|                   2|             2007|             3|             

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2006_data.csv"
spark.sparkContext.addFile(url)
death_2006 = spark.read.csv(SparkFiles.get("2006_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2006_final = data_clean_up2(death_2006)
death_2006_final.show()

+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|icd_death_code|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|race|hispanic|entity_condition_1|entity_condition_2|entity_condition_3|
+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|          J449|       2|        3|             1|  1| 78|             3|                   1|             2006|             3|              7|   1|       6|            11I469|            61J449|            62J984|
|          C509|       1|        4|             1|  2| 73|             4|                   2|             2006|             3|             

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2005_data.csv"
spark.sparkContext.addFile(url)
death_2005 = spark.read.csv(SparkFiles.get("2005_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2005_final = data_clean_up2(death_2005)
death_2005_final.show()

+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|icd_death_code|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|race|hispanic|entity_condition_1|entity_condition_2|entity_condition_3|
+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|          C439|       1|        2|             1|  2| 45|             2|                   2|             2005|             3|              7|   1|       6|            11C439|                 0|                 0|
|          J439|       1|        4|             1|  1| 61|             4|                   7|             2005|             3|             

In [ ]:
# COMBINE ALL DATAFRAMES
#SKIP USING TOO MUCH MEMORY

In [ ]:
death1 = death_2015_final.unionByName(death_2014_final)

In [ ]:
death2 = death1.unionByName(death_2013_final)

In [ ]:
death3 = death2.unionByName(death_2012_final)

In [ ]:
death4 = death3.unionByName(death_2011_final)

In [ ]:
death5 = death4.unionByName(death_2010_final)

In [ ]:
death6 = death5.unionByName(death_2009_final)

In [ ]:
death7 = death6.unionByName(death_2008_final)

In [ ]:
death8 = death7.unionByName(death_2007_final)

In [ ]:
death9 = death8.unionByName(death_2006_final)

In [ ]:
death10 = death9.unionByName(death_2005_final)
death10.count()

9925513

In [ ]:
# death10.show(5)

In [9]:
import pandas as pd

In [10]:
# see the temp code for a bigger break down, getting top 60
codes = death_2015_final.select("*").toPandas()
codes2 = codes.value_counts("icd_death_code")
codes2.to_string()
codes3 = pd.DataFrame(codes2)
codes3.insert(0, 'ID', range(1, 1 +len(codes3)))
codes3 = codes3.reset_index()

In [11]:
df60 = codes3.head(60)
df60 = pd.DataFrame(df60)
code_list_60 = df60["icd_death_code"].tolist()
df55 = codes3.head(55)
df55 = pd.DataFrame(df55)
code_list_55 = df55["icd_death_code"].tolist()
df50 = codes3.head(50)
df50 = pd.DataFrame(df50)
code_list_50 = df50["icd_death_code"].tolist()
df45 = codes3.head(45)
df45 = pd.DataFrame(df45)
code_list_45 = df45["icd_death_code"].tolist()
df40 = codes3.head(40)
df40 = pd.DataFrame(df40)
code_list_40 = df40["icd_death_code"].tolist()
df35 = codes3.head(35)
df35 = pd.DataFrame(df35)
code_list_35 = df35["icd_death_code"].tolist()
df30 = codes.head(30)
df30 = pd.DataFrame(df30)
code_list_30 = df30["icd_death_code"].tolist()
df25 = codes.head(25)
df25 = pd.DataFrame(df25)
code_list_25 = df25["icd_death_code"].tolist()
df20 = codes.head(20)
df20 = pd.DataFrame(df20)
code_list_20 = df20["icd_death_code"].tolist()
df15 = codes.head(15)
df15 = pd.DataFrame(df15)
code_list_15 = df15["icd_death_code"].tolist()
df10 = codes.head(10)
df10 = pd.DataFrame(df10)
code_list_10 = df10["icd_death_code"].tolist()
df5 = codes.head(5)
df5 = pd.DataFrame(df5)
code_list_5 = df5["icd_death_code"].tolist()

The df60 is the top 60 causes of death that affected at least 10,000 people in 2015. The codes are going to be simplified and renamed to smaller categories as there are mutiple that are similar (i.e. several heart conditions or cancer) and thus be slightly more broad. From that list, there are 16 unique codes we will use (one was unknown which we will disregard).
Less might be used if there are issues.  
I251: Heart Failure (coded as 1)  
C349: Cancer (coded as 2)  
J449: COPD (coded as 3)  
G309: Alzheimer's (coded as 4)  
F03: Dementia (coded as 5)  
J189: Pneumonia (coded as 6)  
A419: Sepsis (coded as 7)  
E149: Diabetes (coded as 8)  
G20: Parkinson's (coded as 9)  
X44: Drug exposure  (coded as 10)  
N185: Kidney Disease (coded as 11)  
K746: Cirrhosis of the Liver (coded as 12)  
W19: Falling (coded as 13)  
X74: Self-Harm (coded as 14)  
V892: Auto Accident (coded as 15)   
X95: Assault by Other (coded as 16)

In [12]:
top_list = ["I251","C349","J449","G309","F03","J189","A419","E149","G20","X44","N185","K746","W19","X74","V892","X95"]

In [ ]:
#working with this dataframe
death_2015_final.show(5)

+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|icd_death_code|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|race|hispanic|entity_condition_1|entity_condition_2|entity_condition_3|
+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|          I500|       1|        3|             1|  1| 84|             2|                   1|             2015|             3|              7|   1|       6|            11I500|            61L031|                 0|
|          C349|       1|        6|             1|  1| 70|             2|                   2|             2015|             3|             

In [ ]:
death_2015_final.count()

2239254

In [13]:
#original code was for all dataframes
# death_all_60 = death10.filter(death10.icd_death_code.isin(code_list_60))
# death_all_60.show()

death_all_60 = death_2015_final.filter(death_2015_final.icd_death_code.isin(code_list_60))
# death_all_60.show(5)

In [ ]:
death_all_60.count()

1563030

In [14]:
death_all_60 = death_all_60.na.replace(["E119", "E142", "F019", "I509", "I694","J440", "X42"], ["E149","E149", "F03", "I500", "I64", "J449", "X44"], "icd_death_code")
# death_all_60.count()

In [15]:
# TO GET ALL NUMBERS TO RUN PROPER MUST ANNOTATE EACH ONTO FIRST AND ADD AN ID COLUMN THEN LOAD IN SQL 
from pyspark.sql.functions import monotonically_increasing_id

death_all_final = death_all_60.withColumn("id", monotonically_increasing_id() +1)

In [ ]:
death_all_final.show(5)

+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+---+
|icd_death_code|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|race|hispanic|entity_condition_1|entity_condition_2|entity_condition_3| id|
+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+---+
|          I500|       1|        3|             1|  1| 84|             2|                   1|             2015|             3|              7|   1|       6|            11I500|            61L031|                 0|  1|
|          C349|       1|        6|             1|  1| 70|             2|                   2|             2015|            

In [ ]:
death_all_final.count()

1563030

In [ ]:
death_all_final.printSchema()

root
 |-- icd_death_code: string (nullable = true)
 |-- resident: integer (nullable = true)
 |-- education: integer (nullable = true)
 |-- month_of_death: integer (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- day_of_week_of_death: integer (nullable = true)
 |-- current_data_year: integer (nullable = true)
 |-- injury_at_work: string (nullable = true)
 |-- manner_of_death: integer (nullable = true)
 |-- race: integer (nullable = true)
 |-- hispanic: integer (nullable = true)
 |-- entity_condition_1: string (nullable = false)
 |-- entity_condition_2: string (nullable = false)
 |-- entity_condition_3: string (nullable = false)
 |-- id: long (nullable = false)



In [ ]:
# CONNECT TO DATABASE AND LOAD IN DATA
mode = "append"
jdbc_url="jdbc:postgresql://database-1.ck7rneirj52d.us-east-2.rds.amazonaws.com:5432/Death_Database"
config = {"user":"root", 
          "password": "postgres", 
          "driver":"org.postgresql.Driver"}

In [ ]:
death_all_final.write.jdbc(url=jdbc_url, table='cdc_death_table', mode=mode, properties=config)

In [ ]:
#temporary so I don't have to run the whole thing again when trying to do machine learning

# from pyspark.sql.functions import monotonically_increasing_id
# death_df = data_clean_up(death_2015)
# death_machine = death_df.withColumn("id", monotonically_increasing_id() +1)
# death_machine.show(5)

In [ ]:
# This is my entire 2005-2015 data set, cleaned and encoded, and narrowed down to the top 60 death codes
# THIS IS CURRENTLY ONLY 2015
death_all_final

DataFrame[icd_death_code: string, resident: int, education: int, month_of_death: int, sex: string, age: int, marital_status: string, day_of_week_of_death: int, current_data_year: int, injury_at_work: string, manner_of_death: int, race: int, hispanic: int, entity_condition_1: string, entity_condition_2: string, entity_condition_3: string, id: bigint]

In [16]:
# running out of memory data set too large, going to pick only certain years, if we need to narrow down scope for machine learning, I will narrow down the death codes using the lists above
#change years to update the information

# years = [2015]
# death_machine = death_machine_all.filter(death_machine_all.current_data_year.isin(years))
# death_machine.show(5)

# only using 2015
death_machine_all = death_all_final
# death_machine_all.count()

In [17]:
#already cut down to 60 cut down more
# select which list of codes to use, intervals of 5 between 5 and 60
death_machine_filter = death_machine_all.filter(death_machine_all.icd_death_code.isin(code_list_15))
# death_machine_filter.count()

In [ ]:
death_machine_filter

In [74]:
categories = death_machine_filter.select("*").toPandas()
categories

,icd_death_code,resident,education,month_of_death,sex,age,marital_status,day_of_week_of_death,current_data_year,injury_at_work,manner_of_death,race,hispanic,entity_condition_1,entity_condition_2,entity_condition_3,id
0,I500,1,3,1,1,84,2,1,2015,3,7,1,6,11I500,61L031,0,1
1,C349,1,6,1,1,70,2,2,2015,3,7,1,6,11I469,21R042,31C349,2
2,G309,1,3,1,2,91,3,6,2015,3,7,1,6,11G309,0,0,3
3,I250,1,5,1,2,89,3,5,2015,3,7,1,6,11I250,21S720,61X590,4
4,I500,1,3,1,1,82,2,1,2015,3,7,1,6,11I500,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464075,C349,2,3,12,1,72,1,2,2015,3,7,2,7,11C349,0,0,25769829049
464076,C349,1,3,12,2,58,2,5,2015,3,7,2,7,11I469,21R092,31C349,25769829050
464077,I250,1,3,12,2,56,4,5,2015,3,7,1,6,11I119,12I250,61E669,25769829051
464078,G309,1,1,12,1,68,4,3,2015,3,7,1,2,11I469,21R688,31G309,25769829052


In [75]:
categories2 = categories.value_counts("icd_death_code")
categories2

icd_death_code
C349    126123
I219     92574
G309     84389
I500     59880
I250     55043
I48      19208
W19      14784
I350     12079
dtype: int64

In [18]:
#encoding y values
death_all_coded = death_machine_filter.na.replace(\
['I251', 'C349', 'J449', 'I219', 'G309', 'F03', 'I500', 'I64', 'I250', 'J189', 'C509', 'C259', 'C189', 'A419', 'E149', 'C61', 'I119', 'G20', 'C80', 'I48', 'J440', 'I10', 'X44', 'X42', 'N185', 'J690', 'K746', 'I429', 'C679', 'K703', 'W19', 'C159', 'C719', 'I350', 'E142', 'I469', 'C64', 'J841', 'C56', 'I619', 'I110', 'X74', 'E119', 'F019', 'N390', 'C900', 'X70', 'V892', 'I255', 'X95', 'W18', 'C859', 'I120', 'R99', 'I259', 'I694', 'C169', 'N19', 'C220', 'I509']\
, ["1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36","37","38","39","40","41","42","43","44","45","46","47","48","49","50","51","52","53","54","55","56","57","58","59","60"], "icd_death_code")

In [ ]:
death_all_coded.count()

464080

In [19]:
death_machine = death_all_coded
# death_machine.show(5)

In [ ]:
pd_death_machine = death_machine.select("*").toPandas()
pd_death_machine

In [20]:
###MACHINE LEARNING SET UP VARIABLES
import sklearn
import sklearn.datasets
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [ ]:
#X = death_machine.drop("icd_death_code", axis = 1)
y = death_machine.select("icd_death_code")

In [ ]:
# data_array =  np.array(death_machine.select("resident", "education", "sex", "age", "marital_status", "injury_at_work", "manner_of_death", "race", "hispanic").collect())
# data_array =  np.array(death_machine.select("education", "sex", "age", "marital_status", "injury_at_work", "manner_of_death", "race", "hispanic").collect())
# data_array =  np.array(death_machine.select("education","sex", "age", "marital_status","manner_of_death", "race", "hispanic").collect())
data_array =  np.array(death_machine.select("education","sex", "age", "marital_status","race", "hispanic").collect())
# data_array =  np.array(death_machine.select("sex", "age", "marital_status","race", "hispanic").collect())

In [ ]:
data_array

array([['3', '1', '84', '2', '1', '6'],
       ['6', '1', '70', '2', '1', '6'],
       ['3', '2', '91', '3', '1', '6'],
       ...,
       ['3', '2', '56', '4', '1', '6'],
       ['1', '1', '68', '4', '1', '2'],
       ['9', '1', '83', '5', '2', '7']], dtype='<U21')

In [ ]:
X = data_array

In [ ]:
# X = data_array[:, :-1]
# y = data_array[:,-1]

In [ ]:
X

array([['3', '1', '84', '2', '1', '6'],
       ['6', '1', '70', '2', '1', '6'],
       ['3', '2', '91', '3', '1', '6'],
       ...,
       ['3', '2', '56', '4', '1', '6'],
       ['1', '1', '68', '4', '1', '2'],
       ['9', '1', '83', '5', '2', '7']], dtype='<U21')

In [ ]:
y

array([['6'],
       ['6'],
       ['6'],
       ...,
       ['6'],
       ['2'],
       ['7']], dtype='<U21')

In [ ]:


X = X.astype(str)
# reshape target to be a 2d array
y = y.reshape((len(y), 1))

In [ ]:
print(X.shape, y.shape)

(464080, 6) (464080, 1)


In [ ]:

# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(348060, 6) (348060, 1)
(116020, 6) (116020, 1)


In [ ]:
# already encoded the x values in the cleanup process
# from sklearn.preprocessing import OrdinalEncoder
# xoe = OrdinalEncoder()
# xoe.fit(X)
# X_train_enc = xoe.transform(X_train)
# X_test_enc = xoe.transform(X_test)


In [ ]:
# xoe.set_params()

In [ ]:
# print(X_train, X_test)

In [ ]:
X_train = X_train.astype(int)
X_test = X_test.astype(int)


In [ ]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [ ]:
# did this manually
# yoe = OrdinalEncoder()
# yoe.fit(y)
# y_train_enc = yoe.transform(y_train)
# y_test_enc = yoe.transform(y_test)

In [ ]:
# print(y_train_enc, y_test_enc)

In [21]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

NameError: ignored

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
model = Sequential()

In [ ]:
print(X_train_scaled.shape)
print(X_test_scaled.shape)
print(y_train.shape)
print(y_test.shape)

(348060, 6)
(116020, 6)
(348060, 1)
(116020, 1)


In [ ]:
model.add(Dense(100, activation='relu', input_dim=X_train_scaled.shape[1]))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(y_train.shape[1], activation="softmax"))

In [ ]:
# print(X_train_scaled.shape[1])

In [ ]:
# y_train_enc

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

In [ ]:
model.fit(
    X_train_scaled,
    y_train,
    epochs=10,
    shuffle=True,
    verbose=2
)

Epoch 1/10
10877/10877 - 18s - loss: 0.0000e+00 - accuracy: 0.0226
Epoch 2/10
10877/10877 - 18s - loss: 0.0000e+00 - accuracy: 0.0226
Epoch 3/10
10877/10877 - 18s - loss: 0.0000e+00 - accuracy: 0.0226
Epoch 4/10
10877/10877 - 18s - loss: 0.0000e+00 - accuracy: 0.0226
Epoch 5/10
10877/10877 - 18s - loss: 0.0000e+00 - accuracy: 0.0226
Epoch 6/10
10877/10877 - 18s - loss: 0.0000e+00 - accuracy: 0.0226
Epoch 7/10
10877/10877 - 18s - loss: 0.0000e+00 - accuracy: 0.0226
Epoch 8/10
10877/10877 - 18s - loss: 0.0000e+00 - accuracy: 0.0226
Epoch 9/10
10877/10877 - 17s - loss: 0.0000e+00 - accuracy: 0.0226
Epoch 10/10
10877/10877 - 18s - loss: 0.0000e+00 - accuracy: 0.0226


In [ ]:
# Very Low Accuracy, this is not a good model, going to try a different setup 

In [ ]:
model.save("Death_machine_1.h5")

In [ ]:
# import tensorflow as tf
# from tensorflow import tensorflowjs
# import tensorflowjs as tfjs
!pip install tensorflow
!pip install tensorflowjs


In [ ]:
tfjs.converters.save_keras_model(model, tfjs_target_dir)

NameError: ignored

In [ ]:
# # grid search only? Error no params but I'm not doing grid search
len(model.weights)
# print(model.best_score_)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 100)               700       
_________________________________________________________________
dense_6 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_7 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_8 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 101       
Total params: 31,101
Trainable params: 31,101
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.models import load_model
model_1 = load_model("Death_machine_1.h5")

In [ ]:
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

3626/3626 - 3s - loss: 0.0000e+00 - accuracy: 0.0229
Loss: 0.0, Accuracy: 0.022892605513334274


In [ ]:
test = np.expand_dims(X_test_scaled[0], axis=0)
test.shape

(1, 6)

In [ ]:
test

array([[0.        , 1.        , 0.8173913 , 0.5       , 0.01298701,
        0.75      ]])

In [ ]:
answer = model.predict(test)
answer

array([[1.]], dtype=float32)

In [ ]:
print(f"Precticed class: {np.argmax(model.predict(test), axis=-1)}")

Precticed class: [0]


In [ ]:

input = [1, 2, 3, 4, 5, 6]




In [ ]:
# data = ([0],[input])
# data_array =  np.array(data)
# data_array
# X = data_array.astype(str)
# X.shape
# data = input.astype(int)
# trial.shape

In [ ]:
# data_array =  [(data)]

In [ ]:
x_array = np.asarray(input)

In [ ]:
x_array

array([1, 2, 3, 4, 5, 6])

In [ ]:
x_array.shape

(6,)

In [ ]:
# X = x_array[:, :-1]
# y.reshape((len(y), 1))
data = x_array.reshape(1, (len(x_array)))

In [ ]:
data

array([[1, 2, 3, 4, 5, 6]])

In [ ]:
data.shape

(1, 6)

In [ ]:
results = model_1.predict(data)

In [ ]:
results

array([[1.]], dtype=float32)

In [ ]:
answer = results[0]

In [ ]:
answer

array([1.], dtype=float32)

In [ ]:
testing = model_1.predict(X_test_scaled)

In [ ]:
testing

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)

In [ ]:
testing[0]

array([1.], dtype=float32)

In [ ]:
for i in results:
    x = i[0]
    y = round(x)
    print(y)

1


In [22]:
import sklearn
import sklearn.datasets
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [23]:
pd_death_machine = death_machine.select("*").toPandas()
pd_death_machine

,icd_death_code,resident,education,month_of_death,sex,age,marital_status,day_of_week_of_death,current_data_year,injury_at_work,manner_of_death,race,hispanic,entity_condition_1,entity_condition_2,entity_condition_3,id
0,7,1,3,1,1,84,2,1,2015,3,7,1,6,11I500,61L031,0,1
1,2,1,6,1,1,70,2,2,2015,3,7,1,6,11I469,21R042,31C349,2
2,5,1,3,1,2,91,3,6,2015,3,7,1,6,11G309,0,0,3
3,9,1,5,1,2,89,3,5,2015,3,7,1,6,11I250,21S720,61X590,4
4,7,1,3,1,1,82,2,1,2015,3,7,1,6,11I500,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464075,2,2,3,12,1,72,1,2,2015,3,7,2,7,11C349,0,0,25769829049
464076,2,1,3,12,2,58,2,5,2015,3,7,2,7,11I469,21R092,31C349,25769829050
464077,9,1,3,12,2,56,4,5,2015,3,7,1,6,11I119,12I250,61E669,25769829051
464078,5,1,1,12,1,68,4,3,2015,3,7,1,2,11I469,21R688,31G309,25769829052


In [ ]:
# data_array =  np.array(death_machine.select("resident", "education", "sex", "age", "marital_status", "injury_at_work", "manner_of_death", "race", "hispanic").collect())
# data_array =  np.array(death_machine.select("education", "sex", "age", "marital_status", "injury_at_work", "manner_of_death", "race", "hispanic").collect())
# data_array =  np.array(death_machine.select("education","sex", "age", "marital_status","manner_of_death", "race", "hispanic").collect())
# data_array =  np.array(death_machine.select("education","sex", "age", "marital_status","race", "hispanic").collect())
# data_array =  np.array(death_machine.select("sex", "age", "marital_status","race", "hispanic").collect())

In [ ]:
# X = data_array
# y = death_machine.select("icd_death_code")

In [ ]:
# X = data_array[:, :-1]
# y = data_array[:,-1]

In [ ]:
# print(X.shape, y.shape)

(464080, 4) (464080,)


In [24]:
wanted_data = pd_death_machine[["education","sex", "age", "marital_status","race", "hispanic"]]
wanted_data

,education,sex,age,marital_status,race,hispanic
0,3,1,84,2,1,6
1,6,1,70,2,1,6
2,3,2,91,3,1,6
3,5,2,89,3,1,6
4,3,1,82,2,1,6
...,...,...,...,...,...,...
464075,3,1,72,1,2,7
464076,3,2,58,2,2,7
464077,3,2,56,4,1,6
464078,1,1,68,4,1,2


In [ ]:
X = wanted_data
y = pd_death_machine["icd_death_code"].values.reshape(-1,1)
print(X.shape, y.shape)

(464080, 6) (464080, 1)


In [ ]:
from sklearn.model_selection import train_test_split
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
X_train = X_train.astype(int)
X_test = X_test.astype(int)
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

In [ ]:
from sklearn.linear_model import LinearRegression
model_2 = LinearRegression()
model_2.fit(X_train_scaled, y_train_scaled)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
print(f"Training Data Score: {model_2.score(X_train_scaled, y_train_scaled)}")
print(f"Testing Data Score: {model_2.score(X_test_scaled, y_test_scaled)}")

Training Data Score: 0.03527068224041041
Testing Data Score: 0.03545177966645496


In [ ]:
predictions = model_2.predict(X_test_scaled)
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual labels: {y_test_scaled[:10].tolist()}")

First 10 Predictions:   [[ 0.27265019]
 [ 0.03651411]
 [-0.20603478]
 [ 0.21423094]
 [ 0.08599818]
 [ 0.28964391]
 [ 0.10123074]
 [ 0.20871005]
 [ 0.08111822]
 [-0.03888744]]
First 10 Actual labels: [[1.778933775817609], [1.778933775817609], [-0.6689358004730793], [0.28301347919552167], [0.28301347919552167], [3.682832335154811], [0.011027970718778523], [-0.6689358004730793], [0.011027970718778523], [-0.3969502919963362]]


In [ ]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=.01).fit(X_train_scaled, y_train_scaled)
r2 = ridge.score(X_test_scaled, y_test_scaled)
r2

0.035451779654196325

In [ ]:
import pickle
filename = 'Death_machine_2.sav'
pickle.dump(model_2, open(filename, 'wb'))

# load the model from disk
loaded_model_1 = pickle.load(open(filename, 'rb'))
result = loaded_model_1.score(X_test_scaled, y_test_scaled)
print(result)

0.03545177966645496


In [ ]:
X = wanted_data
y = pd_death_machine["icd_death_code"]
print(X.shape, y.shape)

(464080, 6) (464080,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_train = X_train.astype(int)
X_test = X_test.astype(int)
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [ ]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
model_3 = LogisticRegression(max_iter=1000, verbose =2)
model_3

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=2,
                   warm_start=False)

In [ ]:
model_3.fit(X_train_scaled, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.3min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.3min finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=2,
                   warm_start=False)

In [ ]:
print(f"Training Data Score: {model_3.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model_3.score(X_test_scaled, y_test)}")

Training Data Score: 0.36294317071769233
Testing Data Score: 0.35987760730908464


In [ ]:
predictions = model_3.predict(X_test)
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

First 10 Predictions:   [5 5 5 5 5 5 5 5 5 5]
First 10 Actual labels: [5, 5, 4, 5, 2, 7, 7, 4, 2, 9]


In [ ]:
import pickle
filename = 'Death_machine_3.sav'
pickle.dump(model_3, open(filename, 'wb'))

# load the model from disk
loaded_model_2 = pickle.load(open(filename, 'rb'))
result = loaded_model_2.score(X_test_scaled, y_test)
print(result)

0.35987760730908464


In [ ]:
X = wanted_data
y = pd_death_machine["icd_death_code"]
print(X.shape, y.shape)

(464080, 6) (464080,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train = X_train.astype(int)
X_test = X_test.astype(int)
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.35957593518358905

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.3590070677469402

In [ ]:
feature_names = wanted_data.columns
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.6229105537489908, 'age'),
 (0.1307744684235722, 'education'),
 (0.09371566373260223, 'marital_status'),
 (0.06393353662628881, 'hispanic'),
 (0.04726009039563026, 'sex'),
 (0.04140568707291572, 'race')]

In [ ]:
import pickle
filename = 'Death_machine_4.sav'
pickle.dump(clf, open(filename, 'wb'))

# load the model from disk
loaded_model_3 = pickle.load(open(filename, 'rb'))
result = loaded_model_3.score(X_test, y_test)
print(result)

0.35957593518358905


In [ ]:
import pickle
filename = 'Death_machine_5.sav'
pickle.dump(rf, open(filename, 'wb'))

# load the model from disk
loaded_model_4 = pickle.load(open(filename, 'rb'))
result = loaded_model_4.score(X_test, y_test)
print(result)

0.3590070677469402


In [ ]:
X = wanted_data
y = pd_death_machine["icd_death_code"]
print(X.shape, y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train = X_train.astype(int)
X_test = X_test.astype(int)
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [ ]:
X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
train_scores = []
test_scores = []
for k in range(1, 20, 2):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    train_score = knn.score(X_train_scaled, y_train)
    test_score = knn.score(X_test_scaled, y_test)
    train_scores.append(train_score)
    test_scores.append(test_score)
    print(f"k: {k}, Train/Test Score: {train_score:.3f}/{test_score:.3f}")

k: 1, Train/Test Score: 0.282/0.238
k: 3, Train/Test Score: 0.327/0.297
k: 5, Train/Test Score: 0.335/0.305
k: 7, Train/Test Score: 0.346/0.319
k: 9, Train/Test Score: 0.353/0.326
k: 11, Train/Test Score: 0.357/0.332
k: 13, Train/Test Score: 0.362/0.338
k: 15, Train/Test Score: 0.362/0.338
k: 17, Train/Test Score: 0.364/0.342
k: 19, Train/Test Score: 0.366/0.345


In [ ]:
knn = KNeighborsClassifier(n_neighbors=19)
knn.fit(X_train_scaled, y_train)
print('k=13 Test Acc: %.3f' % knn.score(X_test_scaled, y_test))

k=13 Test Acc: 0.345


In [ ]:
import pickle
filename = 'Death_machine_6.sav'
pickle.dump(knn, open(filename, 'wb'))

# load the model from disk
loaded_model_5 = pickle.load(open(filename, 'rb'))
result = loaded_model_5.score(X_test_scaled, y_test)
print(result)

0.3446647129805206


In [ ]:
X = wanted_data
y = pd_death_machine["icd_death_code"]
print(X.shape, y.shape)

NameError: ignored

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train = X_train.astype(int)
X_test = X_test.astype(int)
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [ ]:

X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
from sklearn.svm import SVC 
model_7 = SVC(kernel='linear')
model_7.fit(X_train_scaled, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
model_7.score(X_test_scaled, y_test)


0.36206688501982415

In [ ]:
from sklearn.metrics import classification_report
predictions = model_7.predict(X_test_scaled)
print(classification_report(y_test, predictions,))

              precision    recall  f1-score   support

           2       0.39      0.83      0.53     31602
           4       0.24      0.00      0.00     23111
           5       0.33      0.67      0.45     20972
           7       0.24      0.10      0.14     14893
           9       0.00      0.00      0.00     13872
          20       0.00      0.00      0.00      4843
          31       0.00      0.00      0.00      3732
          34       0.00      0.00      0.00      2995

    accuracy                           0.36    116020
   macro avg       0.15      0.20      0.14    116020
weighted avg       0.25      0.36      0.25    116020



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import pickle
filename = 'Death_machine_7.sav'
pickle.dump(knn, open(filename, 'wb'))

# load the model from disk
loaded_model_6 = pickle.load(open(filename, 'rb'))
result = loaded_model_6.score(X_test_scaled, y_test)
print(result)

0.23146009308739873


# New Section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
X = wanted_data
y = pd_death_machine["icd_death_code"]
print(X.shape, y.shape)

(464080, 6) (464080,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train = X_train.astype(int)
X_test = X_test.astype(int)
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [ ]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
from sklearn.svm import SVC 
model = SVC(kernel='linear')

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10],
              'gamma': [0.0001, 0.001, 0.01]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [ ]:
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................... C=1, gamma=0.0001, score=0.362, total=71.1min
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 71.1min remaining:    0.0s


In [ ]:
print(grid.best_params_)

In [ ]:
predictions = grid.predict(X_test_scaled)

In [ ]:
print('Test Acc: %.3f' % grid.score(X_test_scaled, y_test))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,))

In [ ]:
import pickle
filename = 'Death_machine_8.sav'
pickle.dump(grid, open(filename, 'wb'))

# load the model from disk
loaded_model_7 = pickle.load(open(filename, 'rb'))
result = loaded_model_7.score(X_test_scaled, y_test)
print(result)

In [41]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [42]:
X = wanted_data
y = pd_death_machine["icd_death_code"]
print(X.shape, y.shape)

(464080, 6) (464080,)


In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [44]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(348060, 6) (348060,)
(116020, 6) (116020,)


In [45]:
X_train = X_train.astype(int)
X_test = X_test.astype(int)
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [46]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [47]:
print(X_train_scaled.shape)
print(X_test_scaled.shape)
print(y_train.shape)
print(y_test.shape)

(348060, 6)
(116020, 6)
(348060,)
(116020,)


In [48]:
from tensorflow.keras.utils import to_categorical
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [49]:
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [50]:
y_train_categorical.shape

(348060, 8)

In [51]:
model4 = Sequential()

In [52]:
model4.add(Dense(units=100, activation='relu', input_dim=6))
model4.add(Dense(units=100, activation='relu'))
model4.add(Dense(units=8, activation='softmax'))

In [53]:
# Compile and fit the model
model4.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [54]:
model4.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               700       
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 808       
Total params: 11,608
Trainable params: 11,608
Non-trainable params: 0
_________________________________________________________________


In [55]:
model4.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
10877/10877 - 13s - loss: 1.6763 - accuracy: 0.3656
Epoch 2/60
10877/10877 - 12s - loss: 1.6629 - accuracy: 0.3710
Epoch 3/60
10877/10877 - 12s - loss: 1.6607 - accuracy: 0.3719
Epoch 4/60
10877/10877 - 12s - loss: 1.6591 - accuracy: 0.3719
Epoch 5/60
10877/10877 - 12s - loss: 1.6580 - accuracy: 0.3727
Epoch 6/60
10877/10877 - 12s - loss: 1.6573 - accuracy: 0.3729
Epoch 7/60
10877/10877 - 12s - loss: 1.6566 - accuracy: 0.3733
Epoch 8/60
10877/10877 - 12s - loss: 1.6561 - accuracy: 0.3738
Epoch 9/60
10877/10877 - 16s - loss: 1.6555 - accuracy: 0.3740
Epoch 10/60
10877/10877 - 12s - loss: 1.6550 - accuracy: 0.3741
Epoch 11/60
10877/10877 - 12s - loss: 1.6546 - accuracy: 0.3740
Epoch 12/60
10877/10877 - 12s - loss: 1.6544 - accuracy: 0.3740
Epoch 13/60
10877/10877 - 12s - loss: 1.6540 - accuracy: 0.3742
Epoch 14/60
10877/10877 - 12s - loss: 1.6538 - accuracy: 0.3738
Epoch 15/60
10877/10877 - 12s - loss: 1.6538 - accuracy: 0.3741
Epoch 16/60
10877/10877 - 12s - loss: 1.6535 - ac

In [57]:
model_loss, model_accuracy = model4.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

3626/3626 - 3s - loss: 1.6552 - accuracy: 0.3714
Normal Neural Network - Loss: 1.65520441532135, Accuracy: 0.3713670074939728


In [58]:
encoded_predictions = model4.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [60]:
encoded_predictions = np.argmax(model4.predict(X_test_scaled[:5]), axis=-1)
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [61]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: [5 5 4 5 2]
Actual Labels: [5, 5, 4, 5, 2]


In [65]:
model4.save("Death_machine_keras4.h5")

In [66]:
from tensorflow.keras.models import load_model
model_1 = load_model("Death_machine_keras4.h5")

In [68]:
model_loss, model_accuracy = model_1.evaluate(X_test_scaled, y_test_categorical, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

3626/3626 - 4s - loss: 1.6552 - accuracy: 0.3714
Loss: 1.65520441532135, Accuracy: 0.3713670074939728


In [ ]:
# icd_death_code
# C349    126123
# I219     92574
# G309     84389
# I500     59880
# I250     55043
# I48      19208
# W19      14784
# I350     12079

In [69]:
model5 = Sequential()

In [70]:
model5.add(Dense(units=100, activation='relu', input_dim=6))
model5.add(Dense(units=100, activation='relu'))
model5.add(Dense(units=1, activation='softmax'))

In [71]:
# Compile and fit the model
model5.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [72]:
model5.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100)               700       
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 101       
Total params: 10,901
Trainable params: 10,901
Non-trainable params: 0
_________________________________________________________________


In [73]:
model5.fit(
    X_train_scaled,
    y_train,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
10877/10877 - 13s - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 2/60


KeyboardInterrupt: ignored